# RFM analysis for e-commerce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import datetime as dt

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Python Datasets/rfm Analysis data.csv',encoding='unicode_escape')

In [5]:
df1=df.copy()

In [ ]:
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


In [6]:
df1.isnull().sum()

,0
Invoice,0
StockCode,0
Description,2928
Quantity,0
InvoiceDate,0
Price,0
Customer ID,107927
Country,0


In [7]:
df1.dropna(inplace=True)

In [8]:
df1=df1[~df1['Invoice'].str.contains('C',na=False)]

In [9]:
df1["TotlaPrice"]=df1["Quantity"]*df1["Price"]

In [ ]:
df1.dtypes

,0
Invoice,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,object
Price,float64
Customer ID,float64
Country,object
TotlaPrice,float64


In [10]:
df1['InvoiceDate']=pd.to_datetime(df1['InvoiceDate'])

In [11]:
# Determining the analysis date for the recency, frquency and monetry
df1['InvoiceDate'].max()
today_date=dt.datetime(2010,12,11)
rfm=df1.groupby('Customer ID').agg({'InvoiceDate':lambda date:(today_date-date.max()).days,
                                     'Invoice':lambda num:len(num),
                                     'TotlaPrice':lambda price:price.sum()})
rfm.columns=['Recency','Frequency','Monetary']

In [12]:
rfm=rfm[rfm['Monetary']>0]

In [13]:
# Date from customer's last purchase. The nearest date gets 5 and the lowest date gets 1
rfm['RecencyScore']=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])

In [14]:
# Total number of purchase. The least frequency gets 1 and the maximun frequency gets 5.
rfm['FrequencyScore']=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[1,2,3,4,5])

In [15]:
# Total spendby the customer.
rfm['MonetaryScore']=pd.qcut(rfm['Monetary'],5,labels=[1,2,3,4,5])

In [16]:
# RFM- The value of 2 variables that were formed was recorded as a RFM_SCORE
rfm['RFM_SCORE']=rfm['RecencyScore'].astype(str)+rfm['FrequencyScore'].astype(str)

In [17]:
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.0,165,33,372.86,2,3,2,23
12347.0,3,71,1323.32,5,4,4,54
12348.0,74,20,222.16,2,2,1,22
12349.0,43,102,2671.14,3,4,5,34
12351.0,11,21,300.93,5,2,2,52
...,...,...,...,...,...,...,...
18283.0,18,230,641.77,4,5,3,45
18284.0,67,28,461.68,3,2,2,32
18285.0,296,12,427.00,1,1,2,11


In [19]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[3-4][3-4]': 'About to Sleep',
    r'[4-5][4-5]': 'Need Attention',
    r'5[0-5]': 'Loyal Customers',
    r'[3-4][1-2]': 'Promising',
    r'5[4-5]': 'Champions',
    r'[1-2][3-4]': 'Potential Loyalists',
    r'[1-2][1-2]': 'New Customers',
    r'[3-4][5-6]': 'Potential'
    }
rfm['Segment']=rfm['RFM_SCORE'].replace(seg_map,regex=True)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,2,23,Potential Loyalists
12347.0,3,71,1323.32,5,4,4,54,Need Attention
12348.0,74,20,222.16,2,2,1,22,New Customers
12349.0,43,102,2671.14,3,4,5,34,About to Sleep
12351.0,11,21,300.93,5,2,2,52,Loyal Customers


In [21]:
rfm[["Segment","Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count","max"]).round()

Recency            Frequency             Monetary        \
                       mean count  max      mean count   max     mean count   
Segment                                                                       
15                    212.0    22  269     203.0    22   366   3086.0    22   
25                    105.0    77  176     226.0    77  1211   2979.0    77   
About to Sleep         40.0   753   71      68.0   753   127   1369.0   753   
Loyal Customers         8.0   269   15      30.0   269    61   1065.0   269   
Need Attention         12.0   876   36     272.0   876  5570   6136.0   876   
New Customers         207.0   997  374      14.0   997    32    458.0   997   
Potential              52.0   128   71     226.0   128   809   3079.0   128   
Potential Loyalists   166.0   607  374      58.0   607   126   1151.0   607   
Promising              42.0   583   71      16.0   583    32    504.0   583   

                               
                          max  
Segment                        
15                    23691.0  
25                    14979.0  
About to Sleep        17671.0  
Loyal Customers       40520.0  
Need Attention       349164.0  
New Customers         26287.0  
Potential             14353.0  
Potential Loyalists   34095.0  
Promising             13151.0